In [1]:
import glob #to read the files
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy.interpolate import interp1d
import os.path 
from tabulate import tabulate #to export in table format
from scipy.interpolate import interp1d
from scipy.optimize import fmin

In [2]:
#kernel = [0.5 * RBF(length_scale=5, length_scale_bounds=(1, 20)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=20, length_scale_bounds=(1, 30)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=1, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=5, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=15, length_scale_bounds=(10, 20)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5))]

In [3]:
kernel = 0.5 * (RBF(length_scale=5, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)))

In [4]:
X_ = np.linspace(-10, 50, 60 + 1)
#xmin = np.linspace(-5, 5, 3)
wave = np.linspace(3000, 9000, 600 + 1)
save_path = '/home/joao/Documentos/PCA'

In [5]:
file = open("treinamento.txt")
lines = file.readlines()
file.close()

In [6]:
training_set = []

In [7]:
for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    training_set.append(co[0])

In [8]:
training_set

['SN2012fr',
 'SN2005cf',
 'SN2013dy',
 'SN2006X',
 'SN2001V',
 'SN2011fe',
 'SN2003du',
 'SN2014J',
 'SN1994D',
 'SN2004dt',
 'SN2002bo',
 'SN1998aq',
 'SN2007le',
 'SN2007af',
 'SN1999aa',
 'SN2000cx',
 'SN2009dc',
 'SN1999ac',
 'LSQ12gdj',
 'SN1999by',
 'SN2008ar',
 'SN2005M',
 'SN2007S',
 'SN2007if',
 'SN2008A',
 'SN2005am',
 'SN2007on',
 'SN2001ep',
 'SN2006D',
 'SN2005ke',
 'SN2004ef',
 'SN2002fk',
 'SN1996X',
 'SN2003kf',
 'SN2008bf',
 'ASASSN-14lp',
 'SN1999dq',
 'SN2012ht',
 'SN2006le',
 'SN1997br',
 'SN2010ae',
 'SN2001eh',
 'SN2006lf',
 'SN2001ay',
 'SN2002dj',
 'SN2008C',
 'SN2003fa',
 'SN2005bl',
 'SN2007hj',
 'SN1994ae',
 'SN2008fp',
 'SN2006S',
 'SN2005na',
 'SN2003W',
 'SN2008s1',
 'SN2006gr',
 'SN2013bh',
 'SN1972E',
 'iPTF13dge',
 'SN2009Y',
 'SN2004as',
 'SN2009ig',
 'SN2008Z',
 'SN1995D',
 'SN2002cd',
 'SN2012cg',
 'SN2005el',
 'SN2007bm',
 'SN2007kk',
 'SN2007F',
 'SN1998bp',
 'SN2006ax',
 'SN1999cl',
 'SN2012hr',
 'SN2006bt',
 'SN1998dh',
 'SN2002cs',
 'iPTF14bdn'

In [9]:
for j in range(0,len(training_set)):
    
    list_data = [[[],[]] for y in range(0,601)]
    
    sn_names=glob.glob("/home/joao/Documentos/templates/data_ext/"+training_set[j]+"*"+".dat")
    
    for p in range(0,len(sn_names)):

        sn_names[p] = sn_names[p][41:]
    
    
    print(training_set[j])
    
    #print(sn_names)
    
    if len(sn_names) > 4:
        
        for i in range(0,len(sn_names)):

            #y = []
            z = []

            file = open(sn_names[i])
            lines = file.readlines()
            file.close()

            for line in lines:
                if line.startswith('#'): continue
                co=line.rstrip().replace('INDEF','Nan').split()

                #y.append(co[0])
                z.append(co[1])
        

            #y = np.array(y, dtype=float)
            z = np.array(z, dtype=float)
            #print(z)
            
            
            for l in range(0,len(z)):
                
                if z[l] < 10**(-64):
                    
                     z[l] = 0
            
            
            timetemp = lines[0]
            timetemp = timetemp[:-1]
            timetemp = float(timetemp[1:])
            
            for jj in range(0,len(z)):
                
                list_data[jj][0].append([timetemp])
                list_data[jj][1].append(z[jj])

            #print(i)
            
        Export = []

        for jj in range(0,len(list_data)):
            
            X_mod = []
            
            Y_mean_mod = []
            
            count = 0
            
            for jjj in range(0,len(list_data[jj][1])):
                
                if list_data[jj][1][jjj] > 10**(-64):
                    
                    count += 1        
                    
            #print(count/len(list_data[jj][1]))
            
            if count > 4:
                
                
                
                for ii in range(0,len(list_data[jj][1])):
                    
                    if list_data[jj][1][ii] > 10**(-64):
                        
                        Y_mean_mod.append(list_data[jj][1][ii])
                        X_mod.append(list_data[jj][0][ii])
                    
                    

                #option = []
                #score = []  

                #print(list_data[jj][1])
                ff = max(Y_mean_mod)
                #print(ff)
                Y_mean_mod = Y_mean_mod/ff

                gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(X_mod, Y_mean_mod)
                y_mean = gp.predict(X_[:, np.newaxis])        
                
                

                for l in range(0,len(y_mean)):

                    Export.append(y_mean[l]*ff)


            else:

                y_meaneff = [0 for y in range(0,61)]
                #print(len(y_mean), len(X_))

                for l in range(0,len(y_meaneff)):

                    Export.append(y_meaneff[l])
                        
             

        table = []
        for l in range(0,len(Export)):
            table.append((Export[l],training_set[j]))

        name_of_file = training_set[j]

        completeName = os.path.join(save_path, name_of_file+".dat")

        f = open(completeName, 'w')
        f.write(tabulate(table, tablefmt="plain"))
        f.close()


SN2012fr
SN2005cf
SN2013dy
SN2006X
SN2001V
SN2011fe
SN2003du
SN2014J
SN1994D
SN2004dt
SN2002bo
SN1998aq
SN2007le
SN2007af
SN1999aa
SN2000cx
SN2009dc
SN1999ac
LSQ12gdj
SN1999by
SN2008ar
SN2005M
SN2007S
SN2007if
SN2008A
SN2005am
SN2007on
SN2001ep
SN2006D
SN2005ke
SN2004ef
SN2002fk
SN1996X
SN2003kf
SN2008bf
ASASSN-14lp
SN1999dq
SN2012ht
SN2006le
SN1997br
SN2010ae
SN2001eh
SN2006lf
SN2001ay
SN2002dj
SN2008C
SN2003fa
SN2005bl
SN2007hj
SN1994ae
SN2008fp
SN2006S
SN2005na
SN2003W
SN2008s1
SN2006gr
SN2013bh
SN1972E
iPTF13dge
SN2009Y
SN2004as
SN2009ig
SN2008Z
SN1995D
SN2002cd
SN2012cg
SN2005el
SN2007bm
SN2007kk
SN2007F
SN1998bp
SN2006ax
SN1999cl
SN2012hr
SN2006bt
SN1998dh
SN2002cs
iPTF14bdn
SN1997bp
SN2004gs
SN2005A
SN2000dk
SN2002de
SN2007qe
SN2015bp
SN2007A
SN1997bq
SN2002cr
SN2002dp
SN2006kf
SN2000cn
SN2007bd
SN2001cp
SN2005kc
SN2006cp
SN2008bc
SN2001bf
SN2002ha
SN2005eu
SN2006gj
SN2008ia
SN2004at
SN1998de
SN2008ha
SN2002fb
SN2005ki
SN2009ad
LSQ12fxd
SN2003cq
SN1995E
LSQ12gpw
SN1997E
SN2005hc